In [ ]:
import glob
import numpy as np
import re
import matplotlib.pyplot as plt


interfaces = [ 1.9199, 2.1319, 2.1999, 2.2599,
               2.2799, 2.2999, 2.3099, 2.3399,
               2.3799, 2.3999, 2.4199, 2.4299, 2.4999,
               2.5999, 4.0000]

ninterfaces = len(interfaces) - 1
maxlag = 50

acf_m = [[] for i in range(ninterfaces)]
acf_p = [[] for i in range(ninterfaces)]

for i in range(ninterfaces):
    
    maxOPvals = []
    pathlengthvals = []
    
    startmove = 600
    endmove = 1001
    
    # Gather information about each move 
    for moveid in range(startmove,endmove):
        
        filen = f'../../{i}/paths/path_gl-{moveid}_ens-{i}.txt'
        
        path = np.genfromtxt(filen,comments=None,dtype=[('pathid','U50'),('time','f8'),('op','f8')],encoding='utf8')
        
        maxOPvals.append(np.max(path['op']))                 # append max op of each path to maxOPvals list
        pathlengthvals.append(len(path))                     # append len of each path 
    
    # Get mean
    maxOPs = np.asarray(maxOPvals)
    pathlens = np.asarray(pathlengthvals)
    OPavg = np.mean(maxOPs) 
    pathlenavg = np.mean(pathlens)
    
    # Get denominator
    denominator_op = np.sum((maxOPs - OPavg)*(maxOPs - OPavg)) 
    denominator_pl = np.sum((pathlens - pathlenavg)*(pathlens - pathlenavg))
    
    # Get numerator
    for lag in range(maxlag):  
        lines = len(maxOPs) - lag                              # tells us about moves after XX lag number
        
        ACFtmp_op = 0.0
        ACFtmp_pl = 0.0
        
        for j in range(lines): # loop over different path/move i
            
            numerator_op = (maxOPs[j] - OPavg)*(maxOPs[j+lag] - OPavg) # maxOPs[j] is max OP at move j or path j
            numerator_pl = (pathlens[j] - pathlenavg)*(pathlens[j+lag] - pathlenavg)
            
            ACFtmp_op = ACFtmp_op + numerator_op/denominator_op
            ACFtmp_pl = ACFtmp_pl + numerator_pl/denominator_pl
        
        acf_m[i].append(ACFtmp_op)
        acf_p[i].append(ACFtmp_pl)
        
# Plot ACF Graphs 

for i in range(ninterfaces):
    plt.plot(range(0,lag+1),acf_m[i], label='Interface {}'.format(i))
    plt.xlabel('Lag')
    plt.ylabel('ACF')
plt.title('Max OP')
plt.legend()
plt.show()

for i in range(ninterfaces):
    plt.plot(range(0,lag+1),acf_p[i], label='Interface {}'.format(i))
    plt.xlabel('Lag')
    plt.ylabel('ACF')
plt.title('Pathlength')
plt.legend()
plt.show()
